## Preprocessing

In [1]:
# Import the required modules
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

# visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold

# Metrics
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve, RocCurveDisplay

# Pre-Processing
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

import tensorflow as tf

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
#checking for nulls/data types
application_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   EIN                     34299 non-null  int64 
 1   NAME                    34299 non-null  object
 2   APPLICATION_TYPE        34299 non-null  object
 3   AFFILIATION             34299 non-null  object
 4   CLASSIFICATION          34299 non-null  object
 5   USE_CASE                34299 non-null  object
 6   ORGANIZATION            34299 non-null  object
 7   STATUS                  34299 non-null  int64 
 8   INCOME_AMT              34299 non-null  object
 9   SPECIAL_CONSIDERATIONS  34299 non-null  object
 10  ASK_AMT                 34299 non-null  int64 
 11  IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 3.1+ MB


In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
#creating a list of columns to drop
cols_to_drop = ['EIN', 'NAME']
application_df.drop(columns = cols_to_drop, inplace =True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [5]:
# Determine the number of unique values in each column.
unique_vals = application_df.nunique()
unique_vals

,0
APPLICATION_TYPE,17
AFFILIATION,6
CLASSIFICATION,71
USE_CASE,5
ORGANIZATION,4
STATUS,2
INCOME_AMT,9
SPECIAL_CONSIDERATIONS,2
ASK_AMT,8747
IS_SUCCESSFUL,2


In [6]:
application_df.columns

Index(['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
       'ORGANIZATION', 'STATUS', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS',
       'ASK_AMT', 'IS_SUCCESSFUL'],
      dtype='object')

In [7]:
#looping to look at value counts
cols = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
       'ORGANIZATION', 'STATUS', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS', 'IS_SUCCESSFUL']

for col in cols:
  print(col)
  print(application_df[col].value_counts())
  print()


APPLICATION_TYPE
APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

AFFILIATION
AFFILIATION
Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: count, dtype: int64

CLASSIFICATION
CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

USE_CASE
USE_CASE
Preservation     28095
ProductDev        5671
CommunityServ      384
Heathcare          146
Other                3
Name: count, dtype: int64

ORGANIZATION
ORGANIZATION
Trust           23515
Association     10255
Co-operative      486
Corporation        43
Nam

In [8]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_df.APPLICATION_TYPE.value_counts()

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
T9,156


In [9]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
threshold = 600
app_type_counts = application_df['APPLICATION_TYPE'].value_counts()
application_types_to_replace = app_type_counts[app_type_counts < threshold].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
Other,804
T8,737
T7,725


In [10]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
application_df.CLASSIFICATION.value_counts()

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
...,...
C4120,1
C8210,1
C2561,1


In [11]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_counts = application_df['CLASSIFICATION'].value_counts()
class_counts.loc[class_counts >1]

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
C7000,777
C1700,287
C4000,194
C5000,116


In [12]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
threshold = 500
class_counts = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = class_counts[class_counts < threshold].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
Other,1484
C7000,777


In [13]:
# Look at ORGANIZATION value counts to identify and replace with "Other"  due to small sample size
application_df.ORGANIZATION.value_counts()

,count
ORGANIZATION,
Trust,23515
Association,10255
Co-operative,486
Corporation,43


In [14]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `orgs_to_replace`
threshold = 12000
org_counts = application_df['ORGANIZATION'].value_counts()
orgs_to_replace = org_counts[org_counts < threshold].index.tolist()

# Replace in dataframe
for cls in orgs_to_replace:
    application_df['ORGANIZATION'] = application_df['ORGANIZATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['ORGANIZATION'].value_counts()

,count
ORGANIZATION,
Trust,23515
Other,10784


In [15]:
# Look at USE_CASE value counts to identify and replace with "Other" due to small sample size
application_df.USE_CASE.value_counts()

,count
USE_CASE,
Preservation,28095
ProductDev,5671
CommunityServ,384
Heathcare,146
Other,3


In [16]:
# Choose a cutoff value and create a list of USE_CASE to be replaced
# use the variable name `uses_to_replace`
#making two categories due to small sample size
threshold = 6000
use_counts = application_df['USE_CASE'].value_counts()
uses_to_replace = use_counts[use_counts < threshold].index.tolist()

# Replace in dataframe
for cls in uses_to_replace:
    application_df['USE_CASE'] = application_df['USE_CASE'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['USE_CASE'].value_counts()

,count
USE_CASE,
Preservation,28095
Other,6204


In [17]:
# Look at AFFILIATION value counts to identify and replace with "Other" due to small sample size
application_df.AFFILIATION.value_counts()

,count
AFFILIATION,
Independent,18480
CompanySponsored,15705
Family/Parent,64
National,33
Regional,13
Other,4


In [18]:
# Choose a cutoff value and create a list of AFFILIATION to be replaced
# use the variable name `aff_to_replace`
#going to make this just two categories
threshold = 16000
aff_counts = application_df['AFFILIATION'].value_counts()
aff_to_replace = aff_counts[aff_counts < threshold].index.tolist()

# Replace in dataframe
for cls in aff_to_replace:
    application_df['AFFILIATION'] = application_df['AFFILIATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['AFFILIATION'].value_counts()

,count
AFFILIATION,
Independent,18480
Other,15819


In [19]:
# Convert categorical data to numeric with `pd.get_dummies`
df2 = pd.get_dummies(application_df)
df2.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_Independent,AFFILIATION_Other,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_C7000,CLASSIFICATION_Other,USE_CASE_Other,USE_CASE_Preservation,ORGANIZATION_Other,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,True,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,True,False,True,False,True,False,False,False,False,False,False,False,False,True,False
1,1,108590,1,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,True,True,False,False,True,False,False,False,False,False,False,False,True,False
2,1,5000,0,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,True,False,True,False,True,False,False,False,False,False,False,False,False,True,False
3,1,6692,1,False,False,True,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,True,False,True,False,False,True,False,False,False,False,False,False,True,False
4,1,142590,1,False,False,True,False,False,False,False,False,True,False,True,False,False,False,False,False,False,True,False,False,True,False,False,False,True,False,False,False,False,False,True,False


In [20]:
# Split our preprocessed data into our features and target arrays
X = df2.drop('IS_SUCCESSFUL', axis=1)
y = df2['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.25 ,random_state=1, stratify=y)

print(X_train.shape, X_test.shape)

(25724, 34) (8575, 34)


In [21]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train["ASK_AMT"].values.reshape(-1,1))

# Scale the data
X_train_scaled = X_train.copy()
X_train_scaled["ASK_AMT"] = X_scaler.transform(X_train["ASK_AMT"].values.reshape(-1,1))

X_test_scaled = X_test.copy()
X_test_scaled["ASK_AMT"] = X_scaler.transform(X_test["ASK_AMT"].values.reshape(-1,1))

## Compile, Train and Evaluate the Model

## **Model #1**

In [22]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#trying a single layer and a few neurons to feel out where we are with the model

# Creating the Keras Sequential model
nn_model = tf.keras.models.Sequential()

# Add our first Dense layer, including the input layer
nn_model.add(tf.keras.layers.Dense(units=7, activation="relu", input_dim=len(X.columns)))

# Add the output layer that uses a probability activation function
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the Sequential model
nn_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 7)                   │             245 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │               8 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 253 (1012.00 B)

 Trainable params: 253 (1012.00 B)

 Non-trainable params: 0 (0.00 B)

In [23]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [24]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=20)

Epoch 1/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6611 - loss: 0.6299
Epoch 2/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7173 - loss: 0.5757
Epoch 3/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7265 - loss: 0.5622
Epoch 4/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7286 - loss: 0.5590
Epoch 5/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7238 - loss: 0.5621
Epoch 6/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7296 - loss: 0.5520
Epoch 7/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7238 - loss: 0.5599
Epoch 8/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7247 - loss: 0.5560
Epoch 9/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7271 - loss: 0.5548
Epoch 10/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7304 - loss: 0.5531
Epoch 11/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7328 - loss: 0.5503
Epoch 12/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step

In [25]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 2ms/step - accuracy: 0.7240 - loss: 0.5590
Loss: 0.5590016841888428, Accuracy: 0.7239649891853333


In [26]:
# Export our model to HDF5 file
nn_model.save('AlphabetSoupCharity_1.h5')

## **Model #2**

In [27]:
#trying to add another layer to the nn with more nurons
# also trying a different activation in my second layer

# Creating the Keras Sequential model
nn_model_2 = tf.keras.models.Sequential()

# Add our first Dense layer, including the input layer
nn_model_2.add(tf.keras.layers.Dense(units=30, activation="relu", input_dim=len(X.columns)))

# Add in a second layer just in case
nn_model_2.add(tf.keras.layers.Dense(units=15, activation="tanh"))

# Add the output layer that uses a probability activation function
nn_model_2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the Sequential model
nn_model_2.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                      │ (None, 30)                  │           1,050 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 15)                  │             465 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 1)                   │              16 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,531 (5.98 KB)

 Trainable params: 1,531 (5.98 KB)

 Non-trainable params: 0 (0.00 B)

In [28]:
# Compile the model
nn_model_2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [29]:
# Train the model, keeping the same epochs to see what changes

fit_model = nn_model_2.fit(X_train_scaled, y_train, epochs=20)

Epoch 1/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6688 - loss: 0.6145
Epoch 2/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7300 - loss: 0.5568
Epoch 3/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7267 - loss: 0.5570
Epoch 4/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7281 - loss: 0.5540
Epoch 5/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7316 - loss: 0.5491
Epoch 6/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7314 - loss: 0.5513
Epoch 7/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7320 - loss: 0.5487
Epoch 8/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7283 - loss: 0.5507
Epoch 9/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7246 - loss: 0.5543
Epoch 10/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7268 - loss: 0.5517
Epoch 11/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7313 - loss: 0.5483
Epoch 12/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step

In [30]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model_2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 2ms/step - accuracy: 0.7249 - loss: 0.5551
Loss: 0.5550788044929504, Accuracy: 0.7248979806900024


In [31]:
# Export our model to HDF5 file
# this one was slighlty worse than the first model
nn_model.save('AlphabetSoupCharity_2.h5')

## **Model #3**

In [32]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#adding two two more layers and increassing the nuerons significantly to see if it helps before more data cleaning

# Creating the Keras Sequential model
nn_model_3 = tf.keras.models.Sequential()

# Add our first Dense layer, including the input layer
nn_model_3.add(tf.keras.layers.Dense(units=75, activation="relu", input_dim=len(X.columns)))

# Add in a second layer just in case
nn_model_3.add(tf.keras.layers.Dense(units=50, activation="relu"))

# Add in a third layer just in case
nn_model_3.add(tf.keras.layers.Dense(units=25, activation="relu"))

# Add the output layer that uses a probability activation function
nn_model_3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the Sequential model
nn_model_3.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_5 (Dense)                      │ (None, 75)                  │           2,625 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 50)                  │           3,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 25)                  │           1,275 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 1)                   │              26 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,726 (30.18 KB)

 Trainable params: 7,726 (30.18 KB)

 Non-trainable params: 0 (0.00 B)

In [33]:
# Compile the model
nn_model_3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [34]:
# Train the model, keeping the same epochs to see what changes
#adding way more epochs to see if it helps at all
fit_model = nn_model_3.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6943 - loss: 0.5982
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7257 - loss: 0.5575
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7280 - loss: 0.5549
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7290 - loss: 0.5559
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7285 - loss: 0.5507
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7277 - loss: 0.5516
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7296 - loss: 0.5515
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7323 - loss: 0.5469
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7351 - loss: 0.5428
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7347 - loss: 0.5456
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7367 - loss: 0.5456
Epoch 12/100
804/804 ━━━━━━━━━━━━━━━━━━━━

In [35]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model_3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - 2ms/step - accuracy: 0.7278 - loss: 0.5727
Loss: 0.5726946592330933, Accuracy: 0.7278134226799011


In [36]:
# Export our model to HDF5 file
nn_model.save('AlphabetSoupCharity_3.h5')

## **Model #4**

In [37]:
#need to try dropping some more columns
#making a copy of the df to make sure i dont lose anything
app_copy = application_df.copy()
app_copy.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,Other,Independent,C1000,Other,Other,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Other,1,1-9999,N,108590,1
2,T5,Other,C3000,Other,Other,1,0,N,5000,0
3,T3,Other,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Other,Trust,1,100000-499999,N,142590,1


In [38]:
#looping to look at value counts
cols = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
       'ORGANIZATION', 'STATUS', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS', 'IS_SUCCESSFUL']

for col in cols:
  print(col)
  print(app_copy[col].value_counts())
  print()


APPLICATION_TYPE
APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: count, dtype: int64

AFFILIATION
AFFILIATION
Independent    18480
Other          15819
Name: count, dtype: int64

CLASSIFICATION
CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: count, dtype: int64

USE_CASE
USE_CASE
Preservation    28095
Other            6204
Name: count, dtype: int64

ORGANIZATION
ORGANIZATION
Trust    23515
Other    10784
Name: count, dtype: int64

STATUS
STATUS
1    34294
0        5
Name: count, dtype: int64

INCOME_AMT
INCOME_AMT
0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: count, dtype: int64

SPECIAL_CONSIDERATIONS
SPECIAL_CONSIDERATIONS
N    34272
Y       27
Name:

In [39]:
# Droping special considerations and status due to the severe imbalance to try and imporove the model
cols_to_drop = ['SPECIAL_CONSIDERATIONS', 'STATUS']
app_copy.drop(columns = cols_to_drop, inplace =True)
app_copy.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,Other,Independent,C1000,Other,Other,0,5000,1
1,T3,Independent,C2000,Preservation,Other,1-9999,108590,1
2,T5,Other,C3000,Other,Other,0,5000,0
3,T3,Other,C2000,Preservation,Trust,10000-24999,6692,1
4,T3,Independent,C1000,Other,Trust,100000-499999,142590,1


In [40]:
# Convert categorical data to numeric with `pd.get_dummies`
app_copy_2= pd.get_dummies(app_copy)
app_copy_2.head()

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_Independent,AFFILIATION_Other,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_C7000,CLASSIFICATION_Other,USE_CASE_Other,USE_CASE_Preservation,ORGANIZATION_Other,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,True,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,True,False,True,False,True,False,False,False,False,False,False,False,False
1,108590,1,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,True,True,False,False,True,False,False,False,False,False,False,False
2,5000,0,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,True,False,True,False,True,False,False,False,False,False,False,False,False
3,6692,1,False,False,True,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,True,False,True,False,False,True,False,False,False,False,False,False
4,142590,1,False,False,True,False,False,False,False,False,True,False,True,False,False,False,False,False,False,True,False,False,True,False,False,False,True,False,False,False,False,False


In [41]:
# Split our preprocessed data into our features and target arrays
X = app_copy_2.drop('IS_SUCCESSFUL', axis=1)
y = app_copy_2['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.25 ,random_state=1, stratify=y)

print(X_train.shape, X_test.shape)

(25724, 31) (8575, 31)


In [42]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train["ASK_AMT"].values.reshape(-1,1))

# Scale the data
X_train_scaled = X_train.copy()
X_train_scaled["ASK_AMT"] = X_scaler.transform(X_train["ASK_AMT"].values.reshape(-1,1))

X_test_scaled = X_test.copy()
X_test_scaled["ASK_AMT"] = X_scaler.transform(X_test["ASK_AMT"].values.reshape(-1,1))

In [43]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#model 3 didnt have enough nuerons, adding even more

# Creating the Keras Sequential model
nn_model_4 = tf.keras.models.Sequential()

# Add our first Dense layer, including the input layer
nn_model_4.add(tf.keras.layers.Dense(units=100, activation="relu", input_dim=len(X.columns)))

# Add in a second layer just in case
nn_model_4.add(tf.keras.layers.Dense(units=50, activation="relu"))

# Add in a third layer just in case
nn_model_4.add(tf.keras.layers.Dense(units=25, activation="relu"))

# Add in a FOURTH layer just in case
nn_model_4.add(tf.keras.layers.Dense(units=15, activation="relu"))

# Add the output layer that uses a probability activation function
nn_model_4.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the Sequential model
nn_model_4.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                      │ (None, 100)                 │           3,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 50)                  │           5,050 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 25)                  │           1,275 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 15)                  │             390 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 1)                   │              16 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 9,931 (38.79 KB)

 Trainable params: 9,931 (38.79 KB)

 Non-trainable params: 0 (0.00 B)

In [44]:
# Compile the model
nn_model_4.compile(loss="binary_crossentropy", optimizer="Adam", metrics=["accuracy"])

In [45]:
# Train the model, keeping the same epochs to see what changes
#adding way more epochs to see if it helps at all
fit_model = nn_model_4.fit(X_train_scaled, y_train, epochs=500)

Epoch 1/500
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7018 - loss: 0.5939
Epoch 2/500
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7309 - loss: 0.5538
Epoch 3/500
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7315 - loss: 0.5502
Epoch 4/500
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7339 - loss: 0.5467
Epoch 5/500
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7344 - loss: 0.5443
Epoch 6/500
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7254 - loss: 0.5557
Epoch 7/500
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7358 - loss: 0.5432
Epoch 8/500
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7282 - loss: 0.5494
Epoch 9/500
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7359 - loss: 0.5426
Epoch 10/500
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7326 - loss: 0.5465
Epoch 11/500
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7351 - loss: 0.5425
Epoch 12/500
804/804 ━━━━━━━━━━━━━━━━━━━━

In [46]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model_4.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - 2ms/step - accuracy: 0.7278 - loss: 0.6206
Loss: 0.6205642819404602, Accuracy: 0.7278134226799011


In [47]:
# Export our model to HDF5 file
nn_model.save('AlphabetSoupCharity_4.h5')